In [10]:
!git clone https://github.com/krishnaik06/Keras-Tuner.git

fatal: destination path 'Keras-Tuner' already exists and is not an empty directory.


In [11]:
!pip install -q -U keras-tuner

In [12]:
import pandas as pd
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch
from tensorflow.keras import Sequential

In [13]:
df = pd.read_csv('/content/Keras-Tuner/Real_Combine.csv')

In [14]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [15]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
Y = Y.fillna(0)

In [24]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(
        optimizer=Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [25]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project1',
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Oracle from existing project project1/Air Quality Index/oracle.json


In [26]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [29]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 11s]
val_mean_absolute_error: 63.357994079589844

Best val_mean_absolute_error So Far: 62.449273427327476
Total elapsed time: 00h 00m 32s
INFO:tensorflow:Oracle triggered exit


In [30]:
tuner.results_summary()

Results summary
Results in project1/Air Quality Index
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 6
units_0: 192
units_1: 96
learning_rate: 0.001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
Score: 62.449273427327476
Trial summary
Hyperparameters:
num_layers: 5
units_0: 96
units_1: 320
learning_rate: 0.0001
units_2: 480
units_3: 192
units_4: 128
units_5: 320
Score: 63.105326334635414
Trial summary
Hyperparameters:
num_layers: 6
units_0: 288
units_1: 128
learning_rate: 0.0001
units_2: 320
units_3: 512
units_4: 384
units_5: 384
units_6: 288
units_7: 384
units_8: 416
units_9: 128
units_10: 384
units_11: 64
units_12: 512
units_13: 448
units_14: 128
Score: 63.32583236694336
Trial summary
Hyperparameters:
num_layers: 20
units_0: 64
units_1: 192
learning_rate: 0.001
units_2: 128
units_3: 192
units_4: 480
units_5: 480
units_6: 96
units_7: 128
units_8: 192
units_9: 480
units_10: 192
units_11: 416
units_12: 192
u